In [1]:
# https://stackoverflow.com/questions/33537476/mailbox-to-csv-using-python

import sys
import mailbox
import csv
import pandas as pd
import numpy as np
import re
import string
import os
import pickle
import email.utils
import html2text
import pyLDAvis.gensim

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim import corpora,models
from email.header import decode_header

np.random.seed(20180313)
pyLDAvis.enable_notebook()

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
def get_content(part):
    content = ''
    payload = part.get_payload()
    if isinstance(payload, str):
        content += payload
    else:
        for part in payload:
            content += get_content(part)
    return content

In [51]:
datapath = '/Users/mjschillawski/Desktop/Miscellaneous Data/'
files = ['Blog Emails.mbox']

# Cornell.mbox','Data.mbox','Go West, Young Man, Go West.mbox',
# 'Promotions-Outdoor Retailer.mbox','Soccer.mbox','The Thread.mbox','Housing.mbox']

#files = ['Go West, Young Man, Go West.mbox']

content = []
for f in files:
    mbox = mailbox.mbox(datapath+f)
    for message in mbox:
        # https://stackoverflow.com/questions/13337528/rendered-html-to-plain-text-using-python
        try:
            thing = html2text.html2text(get_content(message))
            content.append(thing)
        except:
            pass
    print(len(content))

309


In [44]:
with open("partials.pickle","wb") as file:
    pickle.dump(content,file)

In [3]:
with open("partials.pickle", 'rb') as file_handle:
        content = pickle.load(file_handle)

In [24]:
# https://stackoverflow.com/questions/976882/shuffling-a-list-of-objects
np.random.shuffle(content)

In [52]:
def clean_text(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    
    if pd.isnull(text):
        return []
    
    try:        
# https://stackoverflow.com/questions/21209024/python-regex-remove-all-punctuation-except-hyphen-for-unicode-string
        remove = string.punctuation
        pattern = r"[{}]".format(remove) # create the pattern
        text = re.sub(pattern,"",text)

        # break text into words
        tokenizer = RegexpTokenizer(r'\w+')
        processed_text = tokenizer.tokenize(text)
        
        # drop non-words
        processed_text = [text for text in processed_text if len(text) < 25]

        # remove stop words
        processed_text = [text.lower() for text in processed_text if text.lower() 
                          not in stopwords.words('english')]

        # remove other email words
        processed_text = [text.lower() for text in processed_text if text.lower()
                          not in ('re','fwd','r','n','t')]

        # reduce words to stems
        #porter_stemmer = PorterStemmer()
        #processed_text = [porter_stemmer.stem(text) for text in processed_text]

        return processed_text

    except:
        return []

In [53]:
clean_subjects = [cs for cs in [clean_text(c) for c in content] if len(cs) > 0]

In [54]:
with open("partials/cleaned.pickle","wb") as file:
    pickle.dump(clean_subjects,file)

In [55]:
clean_subjects[34]

['view',
 'email',
 'web',
 'page',
 'go',
 'following',
 'address',
 'httpgona',
 'whats',
 'news',
 'nj',
 'starr',
 'hopkins',
 'announced',
 'sunday',
 'wi',
 'challenge',
 'sen',
 'bob',
 'menendez',
 'whoc2a0faces',
 'retrialc2a0on',
 'crimi',
 'nal',
 'corruption',
 'charges',
 'senate',
 'ethics',
 'committee',
 'investigationc2a0i',
 'june',
 '5',
 'primaryc2a0hopkins',
 'worked',
 'onc2a0hillary',
 'clintons',
 'ca',
 'mpaign',
 'served',
 'cable',
 'news',
 'commentator',
 'run',
 'corruption',
 'donald20',
 'trump',
 'excusing',
 'corruption',
 'within',
 'party',
 'nj',
 'advance',
 'media',
 'tx',
 'sen',
 'rep',
 'beto',
 'oe28099rourke',
 'd16',
 'raised',
 '24',
 'million',
 'fourth',
 'quarter',
 'sen',
 'ted',
 'cruz',
 'hauled',
 '19',
 'million',
 'time',
 'st',
 'ill',
 'cruz',
 'e2809cedged',
 'oe28099rourke',
 '2017',
 'calendar',
 'year',
 '71',
 'million',
 '64',
 'millione2809d',
 'mcclatchy',
 'cruz',
 'reported',
 '73',
 'million',
 'bank',
 'tart',
 '2018'

In [56]:
dictionary = corpora.Dictionary(clean_subjects)
corpus = [dictionary.doc2bow(cs) for cs in clean_subjects]

lda = models.ldamodel.LdaModel(corpus,
                              id2word=dictionary,
                              num_topics=15,
                              passes=5,
                              minimum_probability=.05)

In [30]:
with open("clean_subjects3.pickle","wb") as file:
    pickle.dump(clean_subjects,file)
with open('dictionary3.pickle','wb') as file:
    pickle.dump(dictionary,file)
with open('corpus3.pickle','wb') as file:
    pickle.dump(corpus,file)
with open('lda3.pickle','wb') as file:
    pickle.dump(lda,file)

In [65]:
lda.show_topics(num_topics=15)

[(0,
  '0.007*"20" + 0.005*"trump" + 0.003*"state" + 0.003*"new" + 0.003*"e" + 0.002*"would" + 0.002*"primary" + 0.002*"democratic" + 0.002*"voters" + 0.002*"republicans"'),
 (1,
  '0.029*"20" + 0.011*"atlantic" + 0.008*"trump" + 0.008*"09" + 0.007*"new" + 0.006*"president" + 0.004*"email" + 0.004*"washington" + 0.004*"media" + 0.004*"house"'),
 (2,
  '0.004*"20" + 0.002*"new" + 0.002*"former" + 0.001*"state" + 0.001*"republican" + 0.001*"house" + 0.001*"trump" + 0.001*"e280a2" + 0.001*"democratic" + 0.001*"bright"'),
 (3,
  '0.025*"20" + 0.013*"trump" + 0.012*"atlantic" + 0.009*"09" + 0.008*"president" + 0.007*"new" + 0.005*"house" + 0.005*"email" + 0.004*"white" + 0.004*"politics"'),
 (4,
  '0.006*"state" + 0.004*"would" + 0.003*"court" + 0.003*"new" + 0.003*"voting" + 0.003*"elections" + 0.003*"20" + 0.003*"republican" + 0.002*"democratic" + 0.002*"one"'),
 (5,
  '0.119*"20" + 0.007*"sen" + 0.006*"state" + 0.006*"rep" + 0.006*"former" + 0.005*"house" + 0.005*"trump" + 0.005*"gov" + 

In [58]:
pyLDAvis.gensim.prepare(lda,corpus,dictionary)

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      31.378617        1       1 -0.043812 -0.113395
6      24.612516        1       2  0.075245 -0.111180
14      9.821348        1       3  0.073507 -0.102273
3       8.746764        1       4 -0.183589 -0.009622
1       7.340811        1       5 -0.151669 -0.015069
8       6.386382        1       6  0.036356 -0.013233
9       4.318757        1       7 -0.119264  0.039595
7       2.046964        1       8  0.027743  0.173524
11      1.262294        1       9  0.051591  0.049264
13      1.241016        1      10  0.064234 -0.022297
10      1.043087        1      11  0.058038  0.001740
12      0.847903        1      12 -0.004491 -0.019410
0       0.595708        1      13  0.017660  0.031412
4       0.287925        1      14  0.051998  0.044482
2       0.069908        1      15  0.046453  0.066464, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
334    Default  24538.000000            20  24538.000000  30.0000  30.0000
279    Default   3487.000000         state   3487.000000  29.0000  29.0000
299    Default   2751.000000         trump   2751.000000  28.0000  28.0000
903    Default   2493.000000           new   2493.000000  27.0000  27.0000
1029   Default   2269.000000           rep   2269.000000  26.0000  26.0000
1352   Default   1394.000000      atlantic   1394.000000  25.0000  25.0000
2273   Default   1334.000000        e280a2   1334.000000  24.0000  24.0000
985    Default   1844.000000       primary   1844.000000  23.0000  23.0000
1086   Default   1872.000000           sen   1872.000000  22.0000  22.0000
227    Default    766.000000       percent    766.000000  21.0000  21.0000
1259   Default   1741.000000         would   1741.000000  20.0000  20.0000
558    Default   1785.000000    democratic   1785.000000  19.0000  19.0000
685    Default   2264.000000        former   2264.000000  18.0000  18.0000
714    Default   1919.000000           gop   1919.000000  17.0000  17.0000
1907   Default   1579.000000          seat   1579.000000  16.0000  16.0000
755    Default   2102.000000         house   2102.000000  15.0000  15.0000
0      Default    993.000000            09    993.000000  14.0000  14.0000
47     Default   1437.000000          also   1437.000000  13.0000  13.0000
1039   Default   1492.000000    republican   1492.000000  12.0000  12.0000
715    Default   1283.000000           gov   1283.000000  11.0000  11.0000
214    Default   1364.000000           one   1364.000000  10.0000  10.0000
1444   Default    634.000000         court    634.000000   9.0000   9.0000
236    Default   1410.000000     president   1410.000000   8.0000   8.0000
108    Default   1482.000000             e   1482.000000   7.0000   7.0000
251    Default   1198.000000   republicans   1198.000000   6.0000   6.0000
270    Default   1621.000000        senate   1621.000000   5.0000   5.0000
2366   Default    860.000000       however    860.000000   4.0000   4.0000
630    Default    687.000000     elections    687.000000   3.0000   3.0000
101    Default   1397.000000     democrats   1397.000000   2.0000   2.0000
115    Default   1081.000000      election   1081.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
903    Topic15      0.827846           new   2493.693637  -0.7447  -6.1602
12999  Topic15      0.134405         weber      4.893146   3.6710  -7.9782
685    Topic15      0.596506        former   2264.233046  -0.9759  -6.4879
1039   Topic15      0.525816    republican   1492.626781  -0.6854  -6.6141
22928  Topic15      0.131557  prescription      6.180153   3.4161  -7.9996
2273   Topic15      0.465857        e280a2   1334.107972  -0.6941  -6.7352
279    Topic15      0.583876         state   3487.797879  -1.4293  -6.5093
942    Topic15      0.428799         pa

In [59]:
doc_x_topic = np.array([[y for (x,y) in lda[corpus[i]]] for i in range(len(corpus))])

In [63]:
doc_x_topic

array([list([0.06439032, 0.9039945]), list([0.9996092]),
       list([0.99963766]), list([0.75048673, 0.2322187]),
       list([0.9788161]), list([0.9991484]), list([0.9996223]),
       list([0.9996678]), list([0.99978405]), list([0.9837035]),
       list([0.99874717]), list([0.33599424, 0.66000503]),
       list([0.9980716]), list([0.9990505]), list([0.99953634]),
       list([0.9965808]), list([0.55350536, 0.318426, 0.12740086]),
       list([0.9988741]), list([0.49465677, 0.22260636, 0.28254098]),
       list([0.99907774]), list([0.9995984]),
       list([0.43038282, 0.51962197]), list([0.9996413]),
       list([0.9957576]), list([0.99968493]),
       list([0.19616753, 0.17743438, 0.6255851]), list([0.99967617]),
       list([0.105481915, 0.88684696]), list([0.9996256]),
       list([0.98811454]), list([0.7744665, 0.20390913]),
       list([0.99922734]), list([0.99964875]), list([0.9990544]),
       list([0.99967813]), list([0.6281189, 0.13512297, 0.22231403]),
       list([0.999612

In [60]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42, perplexity=30)        # Instantiates Model
tsne_embedding = tsne.fit_transform(doc_x_topic)   # Fits Model/Transforms the Data
tsne_embedding = pd.DataFrame(tsne_embedding, columns=['x','y']) # Creates Pandas Dataframe of TSNE results.
tsne_embedding['hue'] = doc_x_topic.argmax(axis=1) # Creates "hue" based on likeliest topic.

ValueError: setting an array element with a sequence.

In [61]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...

In [ ]:
source = ColumnDataSource(
        data=dict(
            x = tsne_embedding.x,
            y = tsne_embedding.y,
            colors = [all_palettes['Category20'][20][i] for i in tsne_embedding.hue],
            topic = [i for i in tsne_embedding.hue],
            alpha = [0.7] * tsne_embedding.shape[0],
            size = [5] * tsne_embedding.shape[0]
        )
    )

hover_tsne = HoverTool(names=["train"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Topic:</span>
            <span style="font-size: 12px">@topic</span>
        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=700, plot_height=700, tools=tools_tsne, title='Newsgroups')
plot_tsne.circle('x', 'y', size='size', fill_color='colors', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, name="train")

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        alpha[i] = 0.7
        size[i] = 5
    }
    source.trigger('change');
""")

show(plot_tsne)